In [1]:
import pandas as pd
import numpy as np

In [4]:
cf_b = pd.read_csv('https://raw.githubusercontent.com/ecothonCBB/CigaretteButtBins/main/data/%EC%93%B0%EB%A0%88%EA%B8%B0%ED%86%B5_%EC%9C%84%EA%B2%BD%EB%8F%84/%EC%96%91%EC%B2%9C%EA%B5%AC_%EB%8B%B4%EB%B0%B0%EA%BD%81%EC%B4%88%EC%93%B0%EB%A0%88%EA%B8%B0%ED%86%B5_%EC%9C%84%EA%B2%BD%EB%8F%84.csv')
cf_b.head()

,address_name,경도,위도,도로명주소,설치장소,시군구명
0,서울 양천구 목동동로 293,126.875772,37.528178,서울시 양천구 목동동로 293,현대41타워앞 인도,양천구
1,서울 양천구 목동서로 57,126.879863,37.536632,서울시 양천구 목동서로 57,파리지앙A앞 공용주차장,양천구
2,서울 양천구 오목로 232,126.863206,37.525764,서울시 양천구 오목로 232,보성상가옆 먹자골목,양천구
3,서울 양천구 중앙로 276,126.852884,37.520836,서울시 양천구 중앙로 276,신정네거리역 뒤 먹자골목,양천구
4,서울 양천구 오목로 325,126.873627,37.525107,서울시 양천구 오목로 325,농협 옆 골목,양천구


In [7]:
cf_a = pd.read_csv('https://raw.githubusercontent.com/ecothonCBB/CigaretteButtBins/main/data/%E1%84%89%E1%85%A5%E1%84%8B%E1%85%AE%E1%86%AF%E1%84%90%E1%85%B3%E1%86%A8%E1%84%87%E1%85%A7%E1%86%AF%E1%84%89%E1%85%B5_%E1%84%8B%E1%85%A3%E1%86%BC%E1%84%8E%E1%85%A5%E1%86%AB%E1%84%80%E1%85%AE_%E1%84%92%E1%85%B3%E1%86%B8%E1%84%8B%E1%85%A7%E1%86%AB%E1%84%89%E1%85%B5%E1%84%89%E1%85%A5%E1%86%AF%20%E1%84%92%E1%85%A7%E1%86%AB%E1%84%92%E1%85%AA%E1%86%BC_20220801.csv')
cf_a.head()

,자치구,시설 구분,시설형태,설치 위치,규모(제곱미터),설치기관,데이터기준일자,주소
0,양천구,청사,완전개방형,양천구청 부지,8.0,양천구청,2022-08-01,서울특별시 양천구 목동동로 105
1,양천구,청사,완전개방형,해누리타운 4층 옥외정원,8.0,양천구청,2022-08-01,서울특별시 양천구 목동동로 81
2,양천구,청사,완전개방형,양천경찰서 부지,10.0,양천경찰서,2022-08-01,서울특별시 양천구 목동동로 99
3,양천구,청사,개방형,남부지방법원 후문 옆,17.0,남부지방법원,2022-08-01,서울특별시 양천구 신월로 386
4,양천구,청사,개방형,양천세무서 부지 우측,10.0,양천세무서,2022-08-01,서울특별시 양천구 목동동로 165


In [10]:
cf_b[['위도','경도']].duplicated().sum()

0

In [9]:
cf_a = cf_a.drop_duplicates(subset='주소')
cf_a['주소'].duplicated().sum()

0

In [11]:
import json
import requests

def addr_to_lat_lon(addr):
    url = 'https://dapi.kakao.com/v2/local/search/address.json?query={address}'.format(address=addr)
    headers = {"Authorization": "KakaoAK " + '###write your key###'}
    result = json.loads(str(requests.get(url, headers=headers).text))
    match_first = result['documents'][0]['address']
    return float(match_first['y']), float(match_first['x'])

address= cf_a['주소']
print(address.head())

latitude = []
longitude =[]

for i in address:
    try:
        result = addr_to_lat_lon(i)
        latitude.append(result[0])
        longitude.append(result[1])
    except Exception as e:
        print(f"주소 검색 오류: {e}")
        latitude.append(None)  # 또는 다른 기본값 설정
        longitude.append(None)

cf_a['위도'] = latitude
cf_a['경도'] = longitude
cf_a.head()

0    서울특별시 양천구 목동동로 105
1     서울특별시 양천구 목동동로 81
2     서울특별시 양천구 목동동로 99
3     서울특별시 양천구 신월로 386
4    서울특별시 양천구 목동동로 165
Name: 주소, dtype: object
주소 검색 오류: list index out of range


,자치구,시설 구분,시설형태,설치 위치,규모(제곱미터),설치기관,데이터기준일자,주소,위도,경도
0,양천구,청사,완전개방형,양천구청 부지,8.0,양천구청,2022-08-01,서울특별시 양천구 목동동로 105,37.517075,126.866543
1,양천구,청사,완전개방형,해누리타운 4층 옥외정원,8.0,양천구청,2022-08-01,서울특별시 양천구 목동동로 81,37.515976,126.863855
2,양천구,청사,완전개방형,양천경찰서 부지,10.0,양천경찰서,2022-08-01,서울특별시 양천구 목동동로 99,37.516676,126.865594
3,양천구,청사,개방형,남부지방법원 후문 옆,17.0,남부지방법원,2022-08-01,서울특별시 양천구 신월로 386,37.522166,126.863944
4,양천구,청사,개방형,양천세무서 부지 우측,10.0,양천세무서,2022-08-01,서울특별시 양천구 목동동로 165,37.520542,126.869938


In [12]:
cf_a[cf_a['위도'].isnull()]

,자치구,시설 구분,시설형태,설치 위치,규모(제곱미터),설치기관,데이터기준일자,주소,위도,경도
16,양천구,대형건물,개방형,KT 목동타워 부지,3.5,KT 목동타워,2022-08-01,서울특별시 양천구 목동서로 201,NaN,NaN


In [15]:
# 위경도 찾아서 직접 기재

cf_a['위도'][16] = 37.52124412068364
cf_a['경도'][16] = 126.86314260053979
cf_a['위도'].isnull().sum()

<ipython-input-15-93b15ff4f7d9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf_a['위도'][16] = 37.52124412068364
<ipython-input-15-93b15ff4f7d9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cf_a['경도'][16] = 126.86314260053979


0

In [16]:
import folium

# 양천구의 위도
latitude = 37.51423056
longitude = 126.8687083

yc = folium.Map(location=[latitude, longitude],
               zoom_start=14, 
               width=1200, 
               height=700
              )

### 🔷 꽁초 쓰레기통, 🔴 흡연구역

In [17]:
m_map = cf_b[["위도", "경도"]].dropna()
for lat, long in zip(m_map["위도"], m_map["경도"]):
    folium.Marker([lat, long]).add_to(yc)

m_map2 = cf_a[["위도", "경도"]].dropna()
for lat, long in zip(m_map2["위도"], m_map2["경도"]):
    folium.CircleMarker([lat, long], radius=23, color='#cc4831', fill_color='#cc4831').add_to(yc)

yc